## Import Libraries

In [1]:
import pandas as pd
import seaborn as sns

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Q1. Downloading the data

In [3]:
df_jan = pd.read_parquet('./data/yellow_tripdata_2022-01.parquet')

In [5]:
print(len(df_jan.columns))

19


# Q2. Computing duration

In [6]:
df_jan.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


In [8]:
df_jan['duration'] = df_jan.tpep_dropoff_datetime - df_jan.tpep_pickup_datetime
df_jan['duration'] = df_jan.duration.dt.total_seconds() / 60

In [11]:
round(df_jan["duration"].std(),2)

46.45

# Q3. Dropping outliers

In [ ]:
before = len(df_jan)

In [14]:
df_jan = df_jan[(df_jan["duration"] >= 1) & (df_jan["duration"] <= 60)] 
after = len(df_jan)
print(after/before)

0.9827215128995089


# Q4. One-hot encoding

In [23]:
categorical = ['PULocationID', 'DOLocationID']

In [24]:
df_jan[categorical].dtypes

PULocationID    int64
DOLocationID    int64
dtype: object

In [29]:
df_jan[categorical] = df_jan[categorical].astype(str)

In [30]:
train_dicts = df_jan[categorical].to_dict(orient='records')

In [56]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [32]:
X_train.shape

(2421358, 515)

# Q5. Training a model

In [33]:
y_train = df_jan.duration.values

In [34]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)
print(mean_squared_error(y_train, y_pred, squared=False))

6.985303450960275


# Q6. Evaluating the model

In [48]:
df_feb = pd.read_parquet('./data/yellow_tripdata_2022-02.parquet')

In [49]:
df_feb['duration'] = df_feb.tpep_dropoff_datetime - df_feb.tpep_pickup_datetime
df_feb['duration'] = df_feb.duration.dt.total_seconds() / 60
df_feb = df_feb[(df_feb["duration"] >= 1) & (df_feb["duration"] <= 60)] 

In [50]:
df_feb.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,1,2022-02-01 00:06:58,2022-02-01 00:19:24,1.0,5.40,1.0,N,138,252,1,17.0,1.75,0.5,3.90,0.00,0.3,23.45,0.0,1.25,12.433333
1,1,2022-02-01 00:38:22,2022-02-01 00:55:55,1.0,6.40,1.0,N,138,41,2,21.0,1.75,0.5,0.00,6.55,0.3,30.10,0.0,1.25,17.550000
2,1,2022-02-01 00:03:20,2022-02-01 00:26:59,1.0,12.50,1.0,N,138,200,2,35.5,1.75,0.5,0.00,6.55,0.3,44.60,0.0,1.25,23.650000
3,2,2022-02-01 00:08:00,2022-02-01 00:28:05,1.0,9.88,1.0,N,239,200,2,28.0,0.50,0.5,0.00,3.00,0.3,34.80,2.5,0.00,20.083333
4,2,2022-02-01 00:06:48,2022-02-01 00:33:07,1.0,12.16,1.0,N,138,125,1,35.5,0.50,0.5,8.11,0.00,0.3,48.66,2.5,1.25,26.316667


In [57]:
categorical = ['PULocationID', 'DOLocationID']
df_feb[categorical] = df_feb[categorical].astype(str)
train_dicts_feb = df_feb[categorical].to_dict(orient='records')

In [58]:
X_valid = dv.transform(train_dicts_feb)
y_valid = df_feb.duration.values

In [60]:
y_pred = lr.predict(X_valid)

In [61]:
print(mean_squared_error(y_valid, y_pred, squared=False))

7.786576696062271
